In [7]:
contracts = [
{"name": 'c1', "start" : 0, "duration": 2, "price": 10}
,{"name": "c2", "start" :2, "duration" :2, "price" :1}
,{"name": "c3", "start" :2, "duration" :4 , "price": 2}
,{"name": "c4", "start" :4, "duration" :4 , "price": 2}
,{"name": "c5", "start" :8, "duration" :4 , "price": 2}
]

### Step1

Create next hop for each of the contract and also attach price for each so that it's easier to calculate total price later.

In [8]:
def build_neighbour(contracts):
    neigh_set = set({})
    for c in contracts:
        if set(c.keys()) == {'name','start','duration','price'}:
            neig = ()
            end = c['start'] + c['duration']
            for j in contracts:
                if j['start'] >= end:
                    neig = (f"{c['name']}:{c['price']}", f"{j['name']}:{j['price']}")
                    neigh_set.add(neig)
                    
            if not neig:
                neigh_set.add((f"{c['name']}:{c['price']}", None))
        else:
            raise Exception("Missing required keys in provided json body.")
    return neigh_set

This results show contracts c2,c3,c4,c5 can pick up after c1 and so on.

In [9]:
neigh_set = build_neighbour(contracts)
for i in neigh_set:
    print(i)

('c2:1', 'c4:2')
('c1:10', 'c2:1')
('c1:10', 'c5:2')
('c2:1', 'c5:2')
('c1:10', 'c3:2')
('c4:2', 'c5:2')
('c1:10', 'c4:2')
('c5:2', None)
('c3:2', 'c5:2')


### Step2

Create a function that clear out total price and path for each possible path. This function is useful to do the next step.

In [10]:
def calculate_total_profit(path_tuple):
    path = []
    profit = 0
    for ele in path_tuple:
        if ele is not None:
            c, p = ele.split(':')
            path.append(c)
            profit = profit + int(p)
    return (tuple(path), profit)

This function return clear result like showed below.  ('c1:10', 'c2:1', 'c5:2', None) -> (('c1', 'c2', 'c5'), 13)

In [11]:
path_tuple=('c1:10', 'c2:1', 'c5:2', None)
calculate_total_profit(path_tuple)

(('c1', 'c2', 'c5'), 13)

### Step3

The code below generate all the possible contract paths we can pickup.

In [12]:
main_dict = {}
tmp_set = neigh_set
holder_set = neigh_set

while holder_set:
    holder_set = set({})
    for tup1 in tmp_set:
        print(tup1)
        path, profit = calculate_total_profit(tup1)
        main_dict[path] = profit
        
        for tup2 in neigh_set:
            if tup1[-1] == tup2[0]:
                holder_set.add(tup1 + (tup2[-1],))
    
    tmp_set = holder_set

('c2:1', 'c4:2')
('c1:10', 'c2:1')
('c1:10', 'c5:2')
('c2:1', 'c5:2')
('c1:10', 'c3:2')
('c4:2', 'c5:2')
('c1:10', 'c4:2')
('c5:2', None)
('c3:2', 'c5:2')
('c4:2', 'c5:2', None)
('c3:2', 'c5:2', None)
('c1:10', 'c5:2', None)
('c2:1', 'c5:2', None)
('c1:10', 'c3:2', 'c5:2')
('c1:10', 'c2:1', 'c4:2')
('c2:1', 'c4:2', 'c5:2')
('c1:10', 'c4:2', 'c5:2')
('c1:10', 'c2:1', 'c5:2')
('c1:10', 'c3:2', 'c5:2', None)
('c1:10', 'c2:1', 'c4:2', 'c5:2')
('c1:10', 'c4:2', 'c5:2', None)
('c2:1', 'c4:2', 'c5:2', None)
('c1:10', 'c2:1', 'c5:2', None)
('c1:10', 'c2:1', 'c4:2', 'c5:2', None)


Above code will also add the paths and total profit to main_dict.

In [13]:
main_dict

{('c2', 'c4'): 3,
 ('c1', 'c2'): 11,
 ('c1', 'c5'): 12,
 ('c2', 'c5'): 3,
 ('c1', 'c3'): 12,
 ('c4', 'c5'): 4,
 ('c1', 'c4'): 12,
 ('c5',): 2,
 ('c3', 'c5'): 4,
 ('c1', 'c3', 'c5'): 14,
 ('c1', 'c2', 'c4'): 13,
 ('c2', 'c4', 'c5'): 5,
 ('c1', 'c4', 'c5'): 14,
 ('c1', 'c2', 'c5'): 13,
 ('c1', 'c2', 'c4', 'c5'): 15}

Then the last code below is to get the maximum profit from the main_dict and give best_path and max_profit.

In [14]:
best_path = max(main_dict, key=main_dict.get)
max_profit = main_dict[best_path]
print(best_path, max_profit)

('c1', 'c2', 'c4', 'c5') 15
